In [10]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown
from bokeh.models import CustomJS
from bokeh.models import WMTSTileSource
from bokeh.models.widgets import CheckboxButtonGroup
from bokeh.models import OpenURL, TapTool
from bokeh.transform import factor_cmap

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
data = pd.read_csv('F:\Old_Desktop\Data Incubator\landslide.csv')

In [6]:
latlon = data[['country_name', 'latitude', 'longitude', 'landslide_size', 'source_link', 'location_description']]
def wgs84_to_web_mercator(df, lat="latitude", lon="longitude"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    a1 = np.array(df[lon] * (k * np.pi/180.0))
    a2 = np.array(np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k)
    df.loc[:, "latitude"] = a1
    df.loc[:, "longitude"] = a2
    pd.options.mode.chained_assignment = None
    return df

# web mercator coordinates
latlon = wgs84_to_web_mercator(latlon)
max_lat = latlon['latitude'].max()
min_lat = latlon['latitude'].min()
max_lon = latlon['longitude'].max()
min_lon = latlon['longitude'].min()
USA = x_range,y_range = ((min_lat, max_lat), (min_lon,max_lon))

In [7]:
latlon.loc[latlon.landslide_size.isnull(), 'landslide_size'] = 'unknown'

In [8]:
countries = data.country_name.unique().tolist()
countries = [country for country in countries if str(country) != 'nan']
countries.append(' All')
countries.sort()

In [9]:
p = figure(tools='pan, wheel_zoom, tap, reset', x_range=x_range, y_range=y_range, plot_width = 950, 
           title = 'Use wheel zoom and tap on the points for more information, hover for location description')
p.axis.visible = False
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"
p.add_tile(WMTSTileSource(url=url, attribution=attribution))
source = ColumnDataSource(dict(latitude = latlon.latitude, longitude = latlon.longitude, country_name = latlon.country_name,
                              latitude_t = latlon.latitude, longitude_t = latlon.longitude, country_name_t = latlon.country_name,
                              landslide_size = latlon.landslide_size, source_link = latlon.source_link,
                              location_description = latlon.location_description))
cmap = factor_cmap('landslide_size', palette=['#ffffff', '#f7a8e2', '#fcdc12', '#fa774c', '#FC3232', '#040100'], 
                   factors=latlon.landslide_size.unique().tolist())
p.circle(x='latitude', y='longitude', fill_color=cmap, size=8, source = source, legend = 'landslide_size')
p.add_tools(HoverTool(tooltips='@location_description', mode = 'mouse'))
url = "@source_link"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

menu = [(x,x) for x in countries]
def callback(source=source, window=None):
    data = source.data
    f = cb_obj.value
    country_name, latitude, longitude = data['country_name_t'], data['latitude_t'], data['longitude_t']
    if (f != ' All'):
        ind = []
        for i in range(len(country_name)):
            if country_name[i] == f:
                ind.append(i)
        data['country_name'] = []
        data['latitude'] = []
        data['longitude'] = []
    #data['country_name'] = latlon[latlon.country_name == f]['country_name']
        for i in ind:
            data['country_name'][i] = country_name[i]
            data['latitude'][i] = latitude[i]
            data['longitude'][i] = longitude[i]
    else:
        data['country_name'] = data['country_name_t']
        data['latitude'] = data['latitude_t']
        data['longitude'] = data['longitude_t']
    source.change.emit()
    
dropdown = Dropdown(label="Select Country", button_type="warning", menu=menu, callback=CustomJS.from_py_func(callback))
#checkbox = CheckboxButtonGroup(labels = latlon.landslide_size.unique().tolist(), active = [0], 
#                               callback=CustomJS.from_py_func(callback2))
layout = gridplot([[dropdown], [p]], toolbar_location = 'above')
show(layout)

NameError: name 'factor_cmap' is not defined